In [ ]:
'''
Data augmentation, noise addition
Descriptors
Model perfomance
Fine tuning
'''

In [3]:
#for array manipulations
import numpy as np
import pandas as pd
#for image processing
import cv2 
#for displaying images
import matplotlib.pyplot as plt
#to display images in this notebook, not in a separate window
%matplotlib inline
#to access system resources such as directories
import os
#This wilallow us to get the training time of each model
import time

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, auc, classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [4]:
os.getcwd()

'C:\\Users\\USER\\Documents\\GitHub\\maize-disease-detection\\notebooks'

In [5]:
#set our base directory. This should point to the location of the plant-diseases folder
base_dir = 'C:\\Users\\USER\\Documents\\GitHub\\maize-disease-detection'
data_folder = os.path.join(base_dir, 'data')
maize_data_folder = os.path.join(data_folder, 'maize')

In [10]:
#This function loads 32 images of a particular disease
def get_32(disease):
    '''
    disease:
        A string that could be common_rust, healthy, leaf_spot, nothern_leaf_blight
    ........
    disease_images:
        A list of images for the selected disease
    '''
    #this list will contain the 20 images returned
    disease_images = []
    #path to the images
    disease_images_path = os.path.join(maize_data_folder, disease)
    for image_path in os.listdir(disease_images_path):
        image_path = os.path.join(disease_images_path, image_path)
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        disease_images.append(image)
    return disease_images

#This function will help us plot 10 images
def plot_images(images, title):
    '''
    images: List
        List of images
    title: String
        Title for each image i.e name of disease
    '''
    plt.figure(figsize=(15,6))
    for i in range(10):
        plt.subplot(2,5, i+1)
        plt.imshow(images[i])
        plt.title(title)
        plt.xticks([])
        plt.yticks([])
    plt.show()
    
#This function generates ORB features
def extract_features_orb(image, vector_size=32):
    try:
        feature_generator = cv2.ORB_create()
        orb_keypoints = feature_generator.detect(image)
        orb_keypoints = orb_keypoints[:32]
        orb_keypoints, orb_descriptors = feature_generator.compute(image, orb_keypoints)
        orb_descriptors = orb_descriptors.flatten()
        #The descriptor vector size is 128
        needed_size = (vector_size*128)
        if orb_descriptors.size < needed_size:
            #If we have less than 32 keypoints, add zeros to the end of our vector
            orb_descriptors = np.concatenate([orb_descriptors, np.zeros(needed_size - orb_descriptors.size)])
    except cv2.error as e:
        print(f'Error: {e}')
        return None
    return orb_descriptors

#This function generates KAZE features
def extract_features_kaze(image, vector_size=32):
    try:
        feature_generator = cv2.KAZE_create()
        kaze_keypoints = feature_generator.detect(image)
        kaze_keypoints = kaze_keypoints[:32]
        kaze_keypoints, kaze_descriptors = feature_generator.compute(image, kaze_keypoints)
        kaze_descriptors = kaze_descriptors.flatten()
        #The descriptor vector size is 128
        needed_size = (vector_size*128)
        if kaze_descriptors.size < needed_size:
            #If we have less than 32 keypoints, add zeros to the end of our vector
            kaze_descriptors = np.concatenate([kaze_descriptors, np.zeros(needed_size - kaze_descriptors.size)])
    except cv2.error as e:
        print(f'Error: {e}')
        return None
    return kaze_descriptors

def extract_features_hog(image, feature_size=4096):
    hog = cv2.HOGDescriptor()
    features = hog.compute(image)
    required_features = features[:feature_size].ravel()
    return required_features

#Let us extraxt KAZE features
def extract_features(algorithm=0):
    '''
    Algorithm:
        1 for ORB
        0 for KAZE
        2 for HOG
    '''
    #Now let us perform these steps for all the 32 images loaded
    #This will contain all our images
    all_images = []
    #This will contain all our labels
    all_labels = []
    labels = ['common_rust', 'healthy', 'leaf_spot', 'nothern_leaf_blight']
    for i, image_folder in enumerate([common_rust_images, healthy_images, leaf_spot_images, nothern_leaf_blight_images]):
        for image in image_folder:
            all_images.append(image)
            all_labels.append(labels[i])
    features, labels = [], []
    for i, image in enumerate(all_images):
        image_features = []
        try:
            if algorithm == 1:
                image_features = extract_features_orb(image)
            elif algorithm == 0:
                image_features = extract_features_kaze(image)
            else:
                image_features = extract_features_hog(image)
            image_label = all_labels[i]
            features.append(image_features)
            labels.append(image_label)
        except AttributeError as e:
            print(e)
    features = np.array(features)
    labels = np.array(labels)
    features = StandardScaler().fit_transform(features)
    labels = LabelEncoder().fit_transform(labels)
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3)
    
    return X_train, X_test, y_train, y_test

#Let us extraxt KAZE features
def extract_features2(algorithm=0):
    '''
    Algorithm:
        1 for ORB
        0 for KAZE
        2 for HOG
    '''
    #Now let us perform these steps for all the 32 images loaded
    #This will contain all our images
    all_images = []
    #This will contain all our labels
    all_labels = []
    labels = ['common_rust', 'healthy', 'leaf_spot', 'nothern_leaf_blight']
    for i, image_folder in enumerate([common_rust_images, healthy_images, leaf_spot_images, nothern_leaf_blight_images]):
        for image in image_folder:
            all_images.append(image)
            all_labels.append(labels[i])
    features, labels = [], []
    for i, image in enumerate(all_images):
        image_features = []
        try:
            if algorithm == 1:
                image_features = extract_features_orb(image)
            elif algorithm == 0:
                image_features = extract_features_kaze(image)
            else:
                image_features = extract_features_hog(image)
            image_label = all_labels[i]
            features.append(image_features)
            labels.append(image_label)
        except AttributeError as e:
            print(e)
    features = np.array(features)
    labels = np.array(labels)
    features = StandardScaler().fit_transform(features)
    labels = LabelEncoder().fit_transform(labels)
    
    return features, labels

#This method gives us a rough idea about the accuracy of the base models and their raining times
def train_base_models(X_train, y_train, X_test, y_test):
    '''
    '''
    model_accuracy = []
    train_time = []
    model_names = []
    for i, classifier in enumerate(models):
        try:
            #Let us train the model and get the training time
            start_time = time.time()
            classifier.fit(X_train, y_train)
            stop_time = time.time()
            train_time.append(stop_time - start_time)
            predictions = classifier.predict(X_test)
            accuracy = accuracy_score(y_test, predictions)
            model_accuracy.append(round(accuracy, 3))
            model_names.append(names[i])
            print(f'{names[i]}: {round(accuracy, 3)}')
        except Exception as e:
            print(f'Could not train {names[i]} because of {e}')
    df = pd.DataFrame({'Model':model_names, 'Accuracy':model_accuracy, 'Train Time':train_time})
    df = df.sort_values(by=['Accuracy'], ascending=False)
    return df

#This method gives us a rough idea about the accuracy of the base models and their raining times
def train_base_models2(features, labels):
    '''
    '''
    #We use cross-validation
    model_accuracy = []
    train_time = []
    model_names = []
    
    for i, classifier in enumerate(models):
        try:
            #Let us train the model and get the training time
            start_time = time.time()
            scores = cross_val_score(classifier, features, labels, scoring='accuracy', cv=10)
            stop_time = time.time()
            train_time.append(stop_time - start_time)
            model_accuracy.append(round(np.mean(scores), 3))
            model_names.append(names[i])
            print(f'{names[i]}: {round(np.mean(scores), 3)}')
        except Exception as e:
            print(f'Could not train {names[i]} because of {e}')
    df = pd.DataFrame({'Model':model_names, 'Accuracy':model_accuracy, 'Train Time':train_time})
    df = df.sort_values(by=['Accuracy'], ascending=False)
    return df

#This function returns the model perfomance on kaze data
def kaze_results(algorithm=0):
    models_names = []
    model_perfomance = []
    train_time = []
    for i, model in enumerate(kaze_models):
        start = time.time()
        scores = cross_val_score(model, X,y, scoring='accuracy', cv=10)
        stop = time.time()
        models_names.append(model_names[i])
        model_perfomance.append(round(scores.mean(), 2))
        train_time.append((stop - start)/10)
    df = pd.DataFrame({'Model Name':models_names, 'Model Accuracy':model_perfomance, 'Training time':train_time})
    df = df.sort_values(by=['Model Accuracy'], ascending=False)
    return df

#This function returns the model perfomance on hog data
def hog_results(X,y):
    models_names = []
    model_perfomance = []
    train_time = []
    for i, model in enumerate(hog_models):
        start = time.time()
        scores = cross_val_score(model, X, y, scoring='accuracy', cv=10)
        stop = time.time()
        models_names.append(model_names[i])
        model_perfomance.append(round(scores.mean(), 2))
        train_time.append((stop - start)/10)
    df = pd.DataFrame({'Model Name':models_names, 'Model Accuracy':model_perfomance, 'Training Time':train_time})
    df = df.sort_values(by=['Model Accuracy', 'Training Time'], ascending=False)
    return df

#This method generates new features for our training
def get_new_features(model, features):
    model_rfecv = RFECV(estimator=model, step=1, cv=kfold, scoring='accuracy')
    model_results = model_rfecv.fit(features, labels) 
    model_grid_scores = model_rfecv.grid_scores_
    model_ranking = knn_rfecv.ranking_
    model_features_support = model_rfecv.support_
    model_new_features = []
    for i,feature in enumerate(model_features_support):
        if feature:
            model_new_features.append(i)
    model_features = features[:,knn_new_features]
    return model_features

#This method trains a modelon the new features
def train_model(model, features, labels):
    start = time.time()
    scores = cross_val_score(model, features, labels, scoring='accuracy', cv=10)
    stop = time.time()
    t = (stop - start)/10
    return round(scores.mean(), 2), round(t, 4)